In [36]:
import os, sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

import os, librosa, shutil, random
from pydub.utils import make_chunks
from scipy.io import wavfile
from pathlib import Path
from glob import glob
from utils import get_wav_files

In [10]:
base_dir = Path(os.getcwd()).parent
base_dir

PosixPath('/home/kpintaric/LUMEN-Data-Science-IRMAS')

In [59]:
dir = os.path.join(base_dir, "data/raw/IRMAS_Training_Data/")
new_dir = os.path.join(base_dir, "data/processed/IRMAS_Training_Data/")

In [60]:
for d in os.listdir(dir):
    if not os.path.isdir(dir+d): continue
    if os.path.isdir(new_dir+d): continue
    os.makedirs(new_dir+d,exist_ok=True)
    a = 1
    print("Start with folder:",d)
    for f in os.listdir(dir+d):
        wav,sr = librosa.load(dir+d+'/'+f,sr=22050)
        chunks = make_chunks(wav,22050)
        b = 1
        for chunk in chunks:
            wavfile.write(new_dir+d+'/'+f+'_'+str(a)+'_'+str(b)+'_'+d+ ".wav",sr,chunk)
            b += 1
        a += 1
    print("Done with folder:",d)

Start with folder: sax
Done with folder: sax
Start with folder: org
Done with folder: org
Start with folder: gac
Done with folder: gac
Start with folder: pia
Done with folder: pia
Start with folder: gel
Done with folder: gel
Start with folder: voi
Done with folder: voi
Start with folder: cla
Done with folder: cla
Start with folder: cel
Done with folder: cel
Start with folder: vio
Done with folder: vio
Start with folder: tru
Done with folder: tru
Start with folder: flu
Done with folder: flu


In [62]:
dir = os.path.join(base_dir, "data/raw/IRMAS_Training_Data/")
newdir = os.path.join(base_dir, "data/processed/bpm/IRMAS_Training_Data/")
instr = [dir+d for d in os.listdir(dir) if '-' not in d]

In [77]:

def bpm_sync(f1,f2):

    wav,sr = librosa.load(f1,mono=True,sr=22050)
    bpm_base, _ = librosa.beat.beat_track(y=wav, sr=sr)
    print(f1,"- Initial BPM:",round(bpm_base,3))

    wav,sr = librosa.load(f2,mono=True,sr=22050)
    bpm, _ = librosa.beat.beat_track(y=wav, sr=sr)
    wav = librosa.effects.time_stretch(wav, bpm_base/bpm)
    # bpm, _ = librosa.beat.beat_track(y=wav, sr=sr)
    # print(f2,"- Custom  BPM:",round(bpm,3))

    # adjust to 3 sec
    if len(wav)>3*sr: wav = wav[:3*sr]
    else: wav = np.pad(wav,(0,3*sr-len(wav)),'wrap')
    librosa.output.write_wav(d+"temp_file.wav",wav,sr)

    return d+"temp_file.wav"



In [502]:
librosa.feature.rhythm

<module 'librosa.feature.rhythm' from '/home/kpintaric/LUMEN-Data-Science-IRMAS/venv/lib/python3.10/site-packages/librosa/feature/rhythm.py'>

In [65]:
files = get_wav_files(dir)

In [167]:
audio1, sr = librosa.load(files[0], sr=22050)
ipd.Audio(audio1, rate=sr)

In [168]:
audio2, sr = librosa.load(files[1], sr=22050)
ipd.Audio(audio2, rate=sr)

In [86]:


# Determine the pitch difference
12*np.log2(pitch1/pitch2)

# Shift the pitch of the second track
y2_pitch_shifted = librosa.effects.pitch_shift(audio2, sr, pitch_diff)

# Save the modified second track to a file
librosa.output.write_wav('track2_pitch_shifted.wav', y2_pitch_shifted, sr)

TypeError: pitch_shift() takes 1 positional argument but 3 were given

In [145]:
f1 = librosa.pyin(audio1, fmin=50, fmax=2000, sr=sr)[0]
f2 = librosa.pyin(audio2, fmin=50, fmax=2000, sr=sr)[0]

In [146]:
f1_mean = np.nanmean(np.log(f1))
f2_mean = np.nanmean(np.log(f2))

In [148]:
steps = 12*np.log2(f1_mean/f2_mean)

In [151]:
y2_pitch_shifted = librosa.effects.pitch_shift(audio2, sr=sr, n_steps=steps)

In [153]:
ipd.Audio(y2_pitch_shifted, rate=sr)

In [172]:
bpm_base, _ = librosa.beat.beat_track(y=audio1, sr=sr)
bpm, _ = librosa.beat.beat_track(y=audio2, sr=sr)

In [176]:
audio2 = librosa.effects.time_stretch(y=audio2, rate=bpm_base/bpm)

In [178]:
ipd.Audio(audio2, rate=sr)

In [488]:
def sync_pitch_bpm(base_file_path, file_to_convert_path, sr):

    base_file, _ = librosa.load(base_file_path, mono=True, sr=sr)
    bpm_base  = librosa.feature.rhythm.tempo(y=base_file, sr=sr)
    
    file_to_convert, _ = librosa.load(file_to_convert_path, mono=True, sr=sr)
    bpm = librosa.feature.rhythm.tempo(y=file_to_convert, sr=sr)

    bpm_synced = pyrb.time_stretch(file_to_convert, sr=sr, rate=bpm_base[0]/bpm[0])

    f_base = librosa.pyin(base_file, fmin=65, fmax=2093, sr=sr)[0]
    f = librosa.pyin(bpm_synced, fmin=65, fmax=2093, sr=sr)[0]

    if not np.isnan(f).all():
        f_base_mean = np.nanmean(np.log(f_base))
        f_mean = np.nanmean(np.log(f))
        steps = np.round(12*np.log2(np.exp(f_base_mean)/(np.exp(f_mean))),0)
    else:
        steps=0

    pitch_and_bpm_synced = pyrb.pitch_shift(y=bpm_synced, sr=sr, n_steps=steps)

    f_base_start = librosa.onset.onset_detect(y=base_file, units="samples")[0]
    synced_start = librosa.onset.onset_detect(y=pitch_and_bpm_synced, units="samples")[0]
    
    if f_base_start > synced_start:
        diff = f_base_start - synced_start
        synced = np.pad(pitch_and_bpm_synced, (diff,), mode="constant", constant_values=0)
    
    else:
        
        diff = synced_start - f_base_start
        synced = pitch_and_bpm_synced[diff:]
    
    base_file = librosa.util.normalize(base_file)
    synced = librosa.util.normalize(synced)


    return (base_file + np.resize(synced, base_file.shape)) / 2

In [497]:
synced = sync_pitch_bpm(files[789], files[963], sr=22050)

In [496]:
ipd.Audio(synced, rate=22050)

In [493]:
ipd.Audio(librosa.load(files[757])[0], rate=22050)

In [494]:
ipd.Audio(librosa.load(files[57])[0], rate=22050)

In [212]:
sf.write("temp_file.wav", synced, samplerate=22050)

In [217]:
AudioSegment.from_file("temp_file.wav")

In [229]:
def overlay_sound(base_file, overlayed_file):

    base_file = AudioSegment.from_file(base_file)
    overlayed_file = AudioSegment.from_file(overlayed_file)

    return base_file.overlay(overlayed_file, loop=True)

In [230]:
overlay_sound(files[0], "temp_file.wav")

In [238]:
sync_pitch_bpm(files[10], files[50], sr=22050)

array([-0.03458524, -0.06067695, -0.06338196, ..., -0.00751826,
       -0.00867958, -0.00401316], dtype=float32)

In [410]:
c, _ = librosa.load("/home/kpintaric/LUMEN-Data-Science-IRMAS/data/test_sounds/piano-c_C_major.wav")
d, _ = librosa.load("/home/kpintaric/LUMEN-Data-Science-IRMAS/data/test_sounds/piano-d_D_major.wav")

In [413]:
c_f = librosa.pyin(c, fmin=65, fmax=2093, sr=22050)[0]
d_f = librosa.pyin(d, fmin=65, fmax=2093, sr=22050)[0]

In [414]:
d_f

array([         nan, 593.88281644, 593.88281644, 590.46230312,
       590.46230312, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       590.46230312, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       590.46230312, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88281644,
       593.88281644, 593.88281644, 593.88281644, 593.88

In [430]:
f_base_mean = np.nanmean(np.log(c_f))
f_mean = np.nanmean(np.log(d_f))
steps = 12*np.log2(np.exp(f_base_mean)/(np.exp(f_mean)))

In [445]:
pitched = pyrb.pitch_shift(y=d, sr=22050, n_steps=-2)

In [449]:
ipd.Audio(pitched, rate=22050)